In [896]:
import tensorflow as tf
new_model = lambda: tf.keras.Sequential(layers=[
    tf.keras.layers.Conv2D(32, (5,5), input_shape=(200,320,3), activation='relu'),
    tf.keras.layers.MaxPool2D((5,5)),
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.MaxPool2D((5,5)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1000, 'relu', use_bias=False),
    tf.keras.layers.Dense(1000, 'relu', use_bias=False),
    tf.keras.layers.Dense(100, 'relu', use_bias=False),
    tf.keras.layers.Dense(6, 'softmax', use_bias=False),
])

In [897]:
model = new_model()

In [898]:
import pandas as pd
import utils
df = pd.read_csv("teacher_preds.csv")
# df = utils.pull_balanced_preds(df, 1203)
df.shape

(6, 18145)

In [899]:
files = pd.DataFrame([i.split("\\") for i in df.columns.sort_values()])
grouped = files.groupby(1)
groups = [grouped.get_group(i).sort_values(3).iloc[-4:] for i in files[1].unique()]
ends = pd.concat(groups)
files = ['\\'.join(i) for i in ends.to_numpy()]

In [900]:
from sklearn.model_selection import train_test_split
import numpy as np
sample_files = np.random.choice(files, 1203, False)
sample_files = files
train_files, val_files = train_test_split(sample_files, test_size=0.2)
len(sample_files), len(train_files), len(val_files)

(2148, 1718, 430)

In [941]:
from math import ceil
import numpy as np
from PIL import Image
class StudentTrainSequence(tf.keras.utils.Sequence):
    def __init__(self, frame, batch_size) -> None:
        self.frame = frame.copy()
        self.batch_size = batch_size
    def __len__(self):
        return ceil(self.frame.shape[1]/self.batch_size)
    def __getitem__(self, idx):
        low = idx*self.batch_size
        high = min(low + self.batch_size, self.frame.shape[1])
        batch_files = self.frame.columns[low:high]
        batch_y = self.frame.loc[:, batch_files]
        # batch_y = batch_y > 0.5
        # batch_y = batch_y.astype(int)
        batch_y  = batch_y.to_numpy().T
        rotates = np.random.random(len(batch_y)) > 0.5
        batch_y = np.argmax(batch_y, 1)
        batch_y = np.array([(y+3)%6 if r else y for y,r in zip(batch_y, rotates)])
        batch_x = batch_files
        batch_x = [Image.open(f).resize((320,200)) for f in batch_x]
        batch_x = [ im.transpose(Image.ROTATE_180) if r else im for im,r in zip(batch_x,rotates)]
        batch_x = [np.asarray(im, np.float32)/255 for im in batch_x]
        batch_x = np.stack(batch_x)
        # batch_x = [tf.io.read_file(f) for f in batch_x]
        # batch_x = [tf.image.resize(tf.io.decode_png(f), (200,320)) for f in batch_x]
        # batch_x = [tf.image.convert_image_dtype(img, tf.float32) for img in batch_x]
        # batch_x = tf.stack(batch_x)
        return batch_x, batch_y

In [942]:
# train_dfs = {i: df.loc[:, train_files] for i in dfs.keys()}
# val_dfs = {i: df.loc[:, val_files] for i in dfs.keys()}
# train_seq = {i: StudentTrainSequence(train_dfs[i], 100) for i in train_dfs.keys()}
# val_seq = {i: StudentTrainSequence(val_dfs[i], 100) for i in val_dfs.keys()}
train_seq = StudentTrainSequence(df[train_files], 32)
val_seq = StudentTrainSequence(df[val_files], 32)

In [943]:
model = new_model()

In [944]:
model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 196, 316, 32)      2432      
                                                                 
 max_pooling2d_60 (MaxPooli  (None, 39, 63, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_61 (Conv2D)          (None, 35, 59, 32)        25632     
                                                                 
 max_pooling2d_61 (MaxPooli  (None, 7, 11, 32)         0         
 ng2D)                                                           
                                                                 
 flatten_30 (Flatten)        (None, 2464)              0         
                                                                 
 dense_120 (Dense)           (None, 1000)            

In [945]:
model.compile("adam",
              tf.keras.losses.SparseCategoricalCrossentropy(),
              jit_compile=True)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [946]:
hist3 = model.fit(
    train_seq,
    validation_data=val_seq,
    epochs=2,
    callbacks=[callback],
    initial_epoch=0
)

Epoch 1/2
54/54 [==============================] - 100s 2s/step - loss: 1.4379 - val_loss: 1.1210
Epoch 2/2
54/54 [==============================] - 112s 2s/step - loss: 0.9773 - val_loss: 1.0238


In [ ]:
from tqdm.notebook import tqdm
y_sum = None
for x,y in tqdm(StudentTrainSequence(df, 100)):
    y_ = y
    # ysum = np.sum(y_, 0)
    # ysum = np.argmax(y_, 1)
    # ysum = tf.one_hot(ysum, 6)
    ysum = np.sum(ysum, 0)
    print(f"{y_sum} + {ysum}")
    y_sum = ysum if y_sum is None else y_sum + ysum

In [ ]:
record = dict()

In [ ]:
y_sum

In [ ]:
record['Dropout Model'] = y_sum
record

In [1123]:
model(np.random.random((1,200,320,3)))

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[1.3100648e-02, 2.8823983e-04, 8.2235467e-03, 9.7035640e-01,
        1.3503333e-03, 6.6807410e-03]], dtype=float32)>

In [ ]:
model(np.random.random((1,200,320,3)))

<tf.Tensor: shape=(1, 6), dtype=float32, numpy=
array([[2.4511356e-02, 5.8173865e-02, 8.0289298e-01, 3.4369217e-04,
        2.5123553e-04, 1.1382691e-01]], dtype=float32)>

In [ ]:
model.summary()

In [682]:
model.save('student11.keras')

In [601]:
import models.model_student as student
import importlib
importlib.reload(student)

Loading Tensorflow: Done    
loading weights


<module 'models.model_student' from 'd:\\Projects\\Agar.io\\models\\model_student.py'>

In [625]:
import numpy as np
r = np.random.random((1,200,320,3))
print(model(r))
print(student.model(r))

tf.Tensor([[0.51605034 0.520384   0.52464443 0.5213149  0.5281905  0.5168024 ]], shape=(1, 6), dtype=float32)
tf.Tensor([[0.505977   0.5156156  0.5133477  0.52109855 0.51454824 0.51768535]], shape=(1, 6), dtype=float32)


In [450]:
5376/3

1792.0